In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)


# Build Classic Transform Using Least Squares

Use standard field to calculate photometric transform by classic method ("slope of slopes")

In [ ]:
import vso.util
import vso.data
import vso.phot
import numpy as np

IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

layout = vso.util.WorkLayout(WORK_ROOT)


Select night, list of objects, and band pair to analyze

In [ ]:
OBJECTS = ['SA20', 'SA38', 'SA41']
SESSION_TAG='2024/20241006'

# OBJECTS = ['SZ Lyn']
# SESSION_TAG='20250126'

BANDS = ('B', 'V')


Build input data for regression

In [ ]:
import astropy.units as u
from vso.phot import ValErr, ValErrDtype, MagErrDtype
from scipy import stats as sst
from astropy.table import vstack
from vso.phot import ClassicDiffTransform, build_classic_regression_input
from vso.util import MagErr, ValErr, MagErrDtype, ValErrDtype



source = vstack([vso.phot.build_classic_regression_input(layout.get_session(vso.util.Session(tag=SESSION_TAG, name=obj)), BANDS)
               for obj in OBJECTS])
len(source)

In [ ]:
source['err'] = np.sqrt(source['dB']['err']**2 + source['dV']['err']**2 + source['dC']['err']**2 \
                        + source['instr dB']['err']**2 + source['instr dV']['err']**2)
source.sort('err')
data_limit = int(len(source)*.75)
data = source[0:data_limit]
len(data), np.min(data['err']), np.max(data['err'])

Calculate classic transform using ordinary least squares

In [ ]:
import numpy as np
from collections import namedtuple


tr1 = vso.phot.calc_classic_diff_transform(data, BANDS)
tr1

Calculate classic transform using weighted least squares

In [ ]:

tr2 = vso.phot.calc_classic_diff_transform_weighted(data, BANDS)
tr2

Calculate residuals and analyze QQ plots

In [ ]:
import matplotlib.pyplot as plt

def calculate_residual(data, bands, tr):
    A = np.array([data['dC']['mag'], -data['airmass']['val'] * data['dC']['mag']]).T
    res1 = np.dot(A, np.array([tr.Ta.val, tr.ka.val]) ) \
          - data[f'd{bands[0]}']['mag'].value + data[f'instr d{bands[0]}']['mag'].value
    res2 = np.dot(A, np.array([tr.Tb.val, tr.kb.val]) ) \
          - data[f'd{bands[1]}']['mag'].value + data[f'instr d{bands[1]}']['mag'].value
    return res1, res2

res11, res12 = calculate_residual(source, BANDS, tr1)
res21, res22 = calculate_residual(source, BANDS, tr2)


fig = plt.figure(figsize=(12.80, 10.24))
gs = fig.add_gridspec(2, 2)

ax = fig.add_subplot(gs[0, 0])
sst.probplot(res11, dist="norm", plot=ax)
ax.set_title(BANDS[0])

ax = fig.add_subplot(gs[0, 1])
sst.probplot(res12, dist="norm", plot=ax)
ax.set_title(BANDS[1])

ax = fig.add_subplot(gs[1, 0])
sst.probplot(res21, dist="norm", plot=ax)
ax.set_title(BANDS[0] + ' weighted')

ax = fig.add_subplot(gs[1, 1])
sst.probplot(res22, dist="norm", plot=ax)
ax.set_title(BANDS[1] + ' weighted')

plt.tight_layout()
plt.show()


In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8.00, 6.00))
gs = fig.add_gridspec(1, 1)

band = BANDS[0]
ax = fig.add_subplot(gs[0, 0], projection='3d')
ax.set_title(BANDS[0])
sc = ax.scatter(source['dC']['mag'], source[f'd{band}']['mag'], res11, c=source['airmass']['val'], marker='.')
ax.set_xlabel('dC')
ax.set_ylabel(f'd{band}')
ax.set_zlabel('residue')

plt.colorbar(sc, label= 'Air mass')
plt.tight_layout()
plt.show()
